# Loads a dataset, and visualizes cuts for a visual check.

Set start index to where you would like to start.

In [37]:
import functools
import os
from pathlib import Path

import numpy as np
import scipy
from cci.utils import project_dir
import polars as pl
import rerun as rr
from rerun.blueprint import (
    Blueprint,
    BlueprintPanel,
    SelectionPanel,
    TimePanel,
    TimeSeriesView,
    Viewport,
)

START_INDEX = 419

OOCHA_DIR = Path(os.environ["OOCHA_DIR"])
DATASET_FOLDER = project_dir() / "data"
COLOR = {
    "HAS": [82, 239, 153],
    "HVT": [17, 94, 65],
    "HVF": [148, 210, 207],
    "HPE": [44, 69, 125],
    "HPR": [209, 121, 248],
    "DAS": [153, 28, 100],
    "DVF": [172, 130, 180],
    "DVT": [102, 51, 180],
    "DPE": [238, 200, 241],
    "DPR": [41, 140, 192],
    "CAS": [183, 209, 101],
    "CVT": [58, 166, 9],
    "CVF": [44, 245, 43],
    "CPE": [93, 64, 48],
    "CPR": [254, 183, 134],
    "cas": [177, 75, 50],
    "cvt": [251, 45, 76],
    "cvf": [251, 189, 19],
    "cpe": [116, 141, 19],
    "cpr": [37, 128, 254],
    "hun": [246, 18, 168],
    "cun": [194, 24, 241],
    "dfb": [160, 127, 61],
    "hvf": [194, 24, 241],
    "Dvf": [160, 127, 61],
    "hpe": [209, 121, 248],
}


@functools.lru_cache(maxsize=4)
def get_signal(signal_path: Path):
    return scipy.io.loadmat(
        signal_path,
        simplify_cells=True,
    )["SIGNALS"]["ecg_diff"].astype(np.float32)


blueprint = Blueprint(
    Viewport(
        TimeSeriesView(
            name="ECG",
        )
    ),
    BlueprintPanel(expanded=True),
    SelectionPanel(expanded=False),
    TimePanel(expanded=False),
)

Load dataset as an iterator

In [38]:
dfb_df = pl.read_csv(DATASET_FOLDER / "dfb.csv")
iterator = iter(
    dfb_df.with_row_index()
    .select(
        [
            "index",
            "file",
            "EPI",
            "epi1",
            "epi2",
            "epi3",
            "Start",
            "start1",
            "start2",
            "start3",
            "Stop",
            "stop1",
            "stop2",
            "stop3",
        ]
    )
    .rows()
)

[next(iterator) for _ in range(START_INDEX)];

Run to visualize next sample

In [39]:
index, file, epi, epi1, epi2, epi3, start, start1, start2, start3, stop, stop1, stop2, stop3 = next(iterator)
epi2 = f"{epi}"
signal = get_signal(OOCHA_DIR / f"{file}.mat")


def clamp(v, minimum=-4, maximum=4):
    return max(
        minimum,
        min(v, maximum),
    )


stop3 = clamp(stop3, start3 + 9000)

rr.init(f"{index}_{file}", spawn=True, blueprint=blueprint)

tick = 0
for i, (epi, start, stop) in enumerate(
    zip([epi, epi1, epi2, epi3], [start, start1, start2, start3], [stop, stop1, stop2, stop3])
):
    lname = f"{epi}_{i}"
    rr.log(lname, rr.SeriesLine(color=COLOR[epi], name=epi), timeless=True)
    for v in signal[start:stop]:
        rr.set_time_seconds("step", tick)
        rr.log(
            lname,
            rr.Scalar(clamp(v)),
        )
        tick += 1 / 500

[2024-03-27T10:29:08Z INFO  egui_wgpu] There were 2 available wgpu adapters: {backend: Vulkan, device_type: DiscreteGpu, name: "AMD Radeon RX 5700 XT (RADV NAVI10)", driver: "radv", driver_info: "Mesa 23.3.6", vendor: 0x1002, device: 0x731F}, {backend: Vulkan, device_type: Cpu, name: "llvmpipe (LLVM 17.0.6, 256 bits)", driver: "llvmpipe", driver_info: "Mesa 23.3.6 (LLVM 17.0.6)", vendor: 0x10005}
[2024-03-27T10:29:08Z INFO  tracing::span] perform;
[2024-03-27T10:29:08Z INFO  zbus::handshake] write_command; command=Auth(Some(External), Some([49, 48, 48, 48]))
[2024-03-27T10:29:08Z INFO  tracing::span] read_command;
[2024-03-27T10:29:08Z INFO  zbus::handshake] write_command; command=NegotiateUnixFD
[2024-03-27T10:29:08Z INFO  tracing::span] read_command;
[2024-03-27T10:29:08Z INFO  zbus::handshake] write_command; command=Begin
[2024-03-27T10:29:08Z INFO  tracing::span] socket reader;
[2024-03-27T10:29:08Z INFO  tracing::span] perform;
[2024-03-27T10:29:08Z INFO  zbus::handshake] write_co